In [14]:
import pandas as pd
df = pd.read_csv("../Data/lstm.csv")
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [15]:
df.category.unique()

array(['food', 'sports'], dtype=object)

In [16]:
# 데이터에 사용된 중복 없는 전체 단어 갯수를 파악
results = set()
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size

536

In [17]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list() # 길이가 다르므로 numpy사용 못함
paragraphs[0:5]


['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [18]:
from tensorflow import keras

keras.utils.set_random_seed(1) # keras에서 set.seed 한것

In [19]:
# one hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph,vocab_size)\
     for paragraph in paragraphs]
encoded_paragraphs

[[342,
  165,
  382,
  16,
  193,
  462,
  395,
  165,
  268,
  442,
  115,
  113,
  449,
  58,
  289,
  243,
  330,
  344,
  230,
  535,
  431,
  528,
  504,
  72,
  30,
  69,
  209,
  527,
  344,
  102,
  4,
  408,
  47,
  149,
  79,
  293,
  274,
  247,
  51,
  427,
  300,
  209,
  483,
  510,
  177,
  269,
  336,
  462,
  292,
  115,
  521,
  344,
  301],
 [78,
  289,
  243,
  314,
  535,
  427,
  504,
  449,
  207,
  474,
  426,
  134,
  69,
  150,
  423,
  339,
  71,
  124,
  265],
 [96,
  174,
  229,
  401,
  333,
  164,
  69,
  29,
  447,
  64,
  174,
  51,
  511,
  29,
  333,
  164,
  165,
  319,
  15,
  113,
  50,
  340,
  527,
  250,
  397,
  71,
  4,
  344,
  237,
  307,
  462,
  174,
  1,
  69,
  431,
  443,
  344,
  301,
  15,
  288,
  408,
  79,
  392],
 [235,
  262,
  247,
  75,
  481,
  115,
  526,
  69,
  29,
  292,
  69,
  526,
  411,
  115,
  304,
  247,
  289,
  69,
  115,
  411,
  247,
  511,
  162,
  4,
  436,
  185,
  115,
  411,
  361,
  292,
  369,
  466,
  44

In [20]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인
max_length = 0 # 시퀀스 패딩: 문장의 길이를 통일시키기 위해
for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)

91


In [21]:
# 문장마다 단어 갯수가 다르므로 sequence padding을 넣어서 문장의 길이를 동일하게 만듭니다.
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(
    encoded_paragraphs,maxlen = max_length,padding='post')
padded_paragraphs_encoding

array([[342, 165, 382, ...,   0,   0,   0],
       [ 78, 289, 243, ...,   0,   0,   0],
       [ 96, 174, 229, ...,   0,   0,   0],
       ...,
       [ 43, 380, 408, ...,   0,   0,   0],
       [449, 115,  26, ...,   0,   0,   0],
       [468, 527, 115, ...,   0,   0,   0]], dtype=int32)

In [22]:
# 분류항목(food,sports)를 수치로 변경하기
categories = df['category'].to_list()

def category_encode(category):
    if category == 'food':
        return [1,0]
    else:
        return [0,1]

In [23]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [24]:
# Feature 확인
print(encoded_paragraphs[0])

[342, 165, 382, 16, 193, 462, 395, 165, 268, 442, 115, 113, 449, 58, 289, 243, 330, 344, 230, 535, 431, 528, 504, 72, 30, 69, 209, 527, 344, 102, 4, 408, 47, 149, 79, 293, 274, 247, 51, 427, 300, 209, 483, 510, 177, 269, 336, 462, 292, 115, 521, 344, 301]


In [25]:
len(encoded_paragraphs[19])

73

---
### 주제를 분류하는 모델 구현하기

In [26]:
model = keras.Sequential()
# 문맥 생성 단계
model.add(keras.layers.Embedding(vocab_size,5,input_length=max_length)) 
# one hot encoding 만드는 애
# Embedding(중복 없는 전체 단어 갯수,index,가장 간 문장의 단어 갯수 )
# 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력합니다.
model.add(keras.layers.LSTM(64))

# 분류 단계
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(2,activation='softmax')) # 문장이므로 확률값을 출력해야 함

In [30]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

In [31]:
import numpy as np
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [32]:
model.fit(train_X,train_Y,batch_size=10,epochs=50)

Epoch 1/50
2/2 [==============================] - 2s 17ms/step - loss: 0.6940 - accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 0s 28ms/step - loss: 0.6925 - accuracy: 0.5500
Epoch 3/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6925 - accuracy: 0.5500
Epoch 4/50
2/2 [==============================] - 0s 22ms/step - loss: 0.6921 - accuracy: 0.5500
Epoch 5/50
2/2 [==============================] - 0s 33ms/step - loss: 0.6916 - accuracy: 0.6000
Epoch 6/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6911 - accuracy: 0.6000
Epoch 7/50
2/2 [==============================] - 0s 73ms/step - loss: 0.6904 - accuracy: 0.6000
Epoch 8/50
2/2 [==============================] - 0s 29ms/step - loss: 0.6896 - accuracy: 0.6000
Epoch 9/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6886 - accuracy: 0.6000
Epoch 10/50
2/2 [==============================] - 0s 25ms/step - loss: 0.6875 - accuracy: 0.6000
Epoch 11/50
2/2 [============